# Example data anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import sys
sys.path.append('../../../')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

[/home/runner/work/polars/polars/polars/polars-io/src/ndjson/core.rs:162] &data_type = Struct(
    [
        Field {
            name: "Context_Name",
            data_type: LargeUtf8,
            is_nullable: true,
            metadata: {},
        },
        Field {
            name: "Customer_MaritalStatus",
            data_type: LargeUtf8,
            is_nullable: true,
            metadata: {},
        },
        Field {
            name: "Customer_CLV",
            data_type: Int64,
            is_nullable: true,
            metadata: {},
        },
        Field {
            name: "Customer_City",
            data_type: LargeUtf8,
            is_nullable: true,
            metadata: {},
        },
        Field {
            name: "IH_Web_Inbound_Accepted_pxLastGroupID",
            data_type: LargeUtf8,
            is_nullable: true,
            metadata: {},
        },
        Field {
            name: "Decision_Outcome",
            data_type: LargeUtf8,
            is_null

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,str,f64,str,str,bool
"""../../../../da…","""12961163496308…","""18296978850252…",1.2927e19,"""11924169303111…","""85960340173729…",false
"""../../../../da…","""10310119206703…","""15547051511679…",1.4856e19,"""11924169303111…","""17707790312303…",true
"""../../../../da…","""13085005177517…","""15302861055043…",5.6458e17,"""65492076973343…","""85960340173729…",false
"""../../../../da…","""12344384365749…","""15547051511679…",4.0723e18,"""31640018552485…","""85960340173729…",false
"""../../../../da…","""45900351175919…","""18296978850252…",1.4677e19,"""31640018552485…","""21184686688498…",true
"""../../../../da…","""11933134639967…","""15302861055043…",1.6330e19,"""70065544577565…",null,false
"""../../../../da…","""28293723319653…","""15302861055043…",8.7675e18,"""31640018552485…","""21184686688498…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'filename': 'filename',
 'Customer_City': 'PREDICTOR_1',
 'Customer_MaritalStatus': 'PREDICTOR_2',
 'Customer_CLV': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder='../../../../data/',
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""../../../../da…","""17650321825511…","""15397355079522…",1.2927e19,"""FirstMortgage3…","""68271718721183…","""Rejected"""
"""../../../../da…","""17431023896186…","""69997289744987…",1.4856e19,"""FirstMortgage3…","""15680603114415…","""Accepted"""
"""../../../../da…","""72435132115739…","""14003634068616…",5.6458e17,"""MoneyMarketSav…","""68271718721183…","""Rejected"""
"""../../../../da…","""42860641083701…","""69997289744987…",4.0723e18,"""BasicChecking""","""68271718721183…","""Rejected"""
"""../../../../da…","""17191915060762…","""15397355079522…",1.4677e19,"""BasicChecking""","""13630333960240…","""Accepted"""
"""../../../../da…","""62375251320348…","""14003634068616…",1.6330e19,"""UPlusFinPerson…",null,"""Rejected"""
"""../../../../da…","""13519217610183…","""14003634068616…",8.7675e18,"""BasicChecking""","""13630333960240…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""../../../../da…","""81860888557738…","""28049261400585…",1.2927e19,"""FirstMortgage3…","""20815822436116…","""Rejected"""
"""../../../../da…","""15661533355166…","""14999204767419…",1.4856e19,"""FirstMortgage3…","""12431552337379…","""Accepted"""
"""../../../../da…","""81603355992145…","""11523366984605…",5.6458e17,"""MoneyMarketSav…","""20815822436116…","""Rejected"""
"""../../../../da…","""13590962396648…","""14999204767419…",4.0723e18,"""BasicChecking""","""20815822436116…","""Rejected"""
"""../../../../da…","""82287982756045…","""28049261400585…",1.4677e19,"""BasicChecking""","""11391482837705…","""Accepted"""
"""../../../../da…","""54861120345324…","""11523366984605…",1.6330e19,"""UPlusFinPerson…",null,"""Rejected"""
"""../../../../da…","""84296278852023…","""11523366984605…",8.7675e18,"""BasicChecking""","""11391482837705…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

filename=filename
Customer_City=PREDICTOR_1
Customer_MaritalStatus=PREDICTOR_2
Customer_CLV=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str
"""17544083608354…","""11916434688085…",1.2927e19,"""FirstMortgage3…","""11344921472931…","""Rejected"""
"""95955741621750…","""12138857531143…",1.4856e19,"""FirstMortgage3…","""27948114276081…","""Accepted"""
"""40418386206986…","""43162512322770…",5.6458e17,"""MoneyMarketSav…","""11344921472931…","""Rejected"""
"""96094466320148…","""12138857531143…",4.0723e18,"""BasicChecking""","""11344921472931…","""Rejected"""
"""32794588652343…","""11916434688085…",1.4677e19,"""BasicChecking""","""49572518824216…","""Accepted"""
"""16234227883469…","""43162512322770…",1.6330e19,"""UPlusFinPerson…",null,"""Rejected"""
"""76007837241871…","""43162512322770…",8.7675e18,"""BasicChecking""","""49572518824216…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'